# Install

In [ ]:
!pip install -q dwave-ocean-sdk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from google.colab import files
import pickle
from tqdm import tqdm
import time

In [ ]:
from collections import defaultdict
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector as inspector
from dimod import ConstrainedQuadraticModel, CQM, SampleSet, cqm_to_bqm, to_networkx_graph
from dwave.system import LeapHybridCQMSampler
from dimod.vartypes import Vartype
from dimod import Binary, quicksum
from dimod import BinaryQuadraticModel

In [ ]:
from google.colab import userdata

# DWAVE Initializing

In [ ]:
endpoint = 'https://cloud.dwavesys.com/sapi'
token = userdata.get('dwave_leap')

# Import Alpha, Beta and Theta data

## Alpha data

In [ ]:
alpha_data = files.upload()
alpha_file_name = list(alpha_data.keys())[0]
alpha_file_name

In [ ]:
alpha = dict()
with open(alpha_file_name) as f:
    lines = f.readlines()
    for line in lines:
      a,b,c = line.replace('\n','').split(' ')
      a = int(a)
      b = int(b)
      c = int(c)
      alpha[(a-1,b-1)] = c
#alpha

## Beta data

In [ ]:
beta_data = files.upload()
beta_file_name = list(beta_data.keys())[0]
beta_file_name

In [ ]:
n_students = 0
with open(beta_file_name) as f:
    lines = f.readlines()
    first_line = lines[0].split(' ')
    #print(first_line)
    n_features = len(first_line)
    #print(n_features)
    beta = [[] for x in range(n_features)]
    for line in lines:
      n_students +=1
      line = line.replace('\n','').split(' ')
      for i in range(n_features):
        beta[i].append(int(line[i]))

In [ ]:
for i in range(n_students):
  for j in range(n_students):
    if i==j:
      alpha[(i,j)] = 0

In [ ]:
#alpha

## Theta data

In [ ]:
theta_data = files.upload()
theta_file_name = list(theta_data.keys())[0]
theta_file_name

In [ ]:
theta = []
with open(theta_file_name) as f:
    lines = f.readlines()
    for line in lines:
      theta.append(int(line.replace('\n','')))
theta

In [ ]:
beta_max = [theta[x] for x in range(n_features)]
tao_min = theta[-2]
tao_max = theta[-1]

## Check problem inputs

In [ ]:
n_groups = 3

In [ ]:
N,C = range(n_students),range(n_groups)

In [ ]:
print(f'Arrange classroom of {n_students} students, into {n_groups} groups of min group \
size: {tao_min} and max group size: {tao_max}')

# BQM solve function

In [ ]:
def produce_anneal_schedule(anneal_time,pause_duration):

  step = anneal_time//2

  p1 = [0,0]
  p2 = [step,0.5]
  p3 = [step+pause_duration,0.5]
  p4 = [anneal_time + pause_duration,1.0]

  return [p1,p2,p3,p4]

In [ ]:
def solve_r_m_n_sap(alpha,beta,theta,n_students,n_groups,tao_min,tao_max,anneal_time = 20,pause_duration = 0, NUM_READS = 1000,verbose = False):

  if verbose:
    print('Creating model...')

  N,C = range(n_students),range(n_groups)

  LAMBDA = n_groups * tao_max*( tao_max - 1)
  x = np.array([[f'x_{i}_{c}' for c in C] for i in N])

  bqm = BinaryQuadraticModel(Vartype.BINARY)

  # objective function

  for i in N:
    for j in N:
      if i == j or alpha.get((i,j),0) == 0:
        continue
      for c in C:
        var1 = x[i,c]
        var2 = x[j,c]
        cost = -1*alpha.get((i,j),0)
        bqm.add_quadratic(var1,var2,cost)

  # Constraint one single group constraint

  for i in N:
      terms = x[i,:]
      terms = [(x,1) for x in terms]
      bqm.add_linear_equality_constraint(terms = terms, lagrange_multiplier = LAMBDA, constant = -1)
      terms.clear

  # min group size

  for c in C:
      terms = x[:,c]
      terms = [(x,1) for x in terms]
      bqm.add_linear_inequality_constraint(terms = terms, lagrange_multiplier = LAMBDA, lb = tao_min,ub = tao_max,label = 'tao_min')
      terms.clear

  # max group size

  for c in C:
      terms = x[:,c]
      terms = [(x,1) for x in terms]
      bqm.add_linear_inequality_constraint(terms = terms, lagrange_multiplier = LAMBDA, constant = -tao_max,label = 'tao_max')
      terms.clear


  # Beta homegeinity constraints
  group_combs = [(c,cp) for c in C for cp in C if c!=cp]

  for c1,c2 in group_combs:
    for f in range(n_features):

      bf = beta[f]
      B = beta_max[f]

      terms1 = [(x[i,c1],bf[i]) for i in N]
      terms2 = [(x[i,c2],-bf[i]) for i in N]
      terms = terms1 + terms2
      bqm.add_linear_inequality_constraint(terms = terms, lagrange_multiplier = LAMBDA, constant = -B,label = 'homophily_break')

  if verbose:
    print('Solving model...')

  # solve the model
  schedule = produce_anneal_schedule(anneal_time,pause_duration)

  sampler = DWaveSampler(solver='Advantage_system6.4',endpoint= endpoint, token = token)
  if verbose:
    print("Connected to sampler", sampler.solver.name)

  sampler = EmbeddingComposite(sampler)
  sampler_name = sampler.properties['child_properties']['chip_id']

  CHAIN_S = 2*LAMBDA

  instance_name = f'CGFP_BQM_{n_students}_{n_groups}_{anneal_time}_{pause_duration}'
  if verbose:
    print(f'Instance name: {instance_name}')
  if pause_duration == 0:
    sampleset = sampler.sample(bqm, num_reads= NUM_READS,annealing_time=anneal_time,label = instance_name,chain_strength=CHAIN_S)
    if verbose:
      print("No Pause")
  else:
    if verbose:
      print(f"With Pause, Schedule {schedule}")

      plt.figure(1, figsize=(3, 3))
      plt.plot(*np.array(schedule).T)
      plt.title("Anneal Schedule")
      plt.xlabel("Time [us]")
      plt.ylabel("Annealing Parameter s")
      plt.ylim([0.0,1.0])
      plt.show()

    sampleset = sampler.sample(bqm, num_reads= NUM_READS,label=instance_name,chain_strength=CHAIN_S,anneal_schedule=schedule)
  if verbose:
    print("Solved")

  return sampleset

In [ ]:
# sampleset_test = solve_r_m_n_sap(alpha,
#                                 beta,
#                                 theta,
#                                 n_students,
#                                 n_groups,
#                                 tao_min,
#                                 tao_max,
#                                 anneal_time = 20,
#                                 pause_duration = 50,
#                                 NUM_READS = 1000,
#                                 verbose = True)

# testing multiple running times and anneal schedules

In [ ]:
anneal_times = [10,20,50,100,200]
pause_durations = [0,10,20,50,100]

NUM_READS = 1000

outputs = []

total_iterations = len(anneal_times)*len(pause_durations)

with tqdm(total=total_iterations) as pbar:

  for anneal_time in anneal_times:
    for pause_duration in pause_durations:
      sampleset = solve_r_m_n_sap(alpha,
                                beta,
                                theta,
                                n_students,
                                n_groups,
                                tao_min,
                                tao_max,
                                anneal_time = anneal_time,
                                pause_duration = pause_duration,
                                NUM_READS = NUM_READS,
                                verbose = False)

      row = {'n':n_students,'c':n_groups,'anneal_time':anneal_time,'pause_duration':pause_duration,'sampleset':sampleset.to_serializable()}
      outputs.append(row)
      time.sleep(5)
      pbar.update(1)

In [ ]:
with open(f"Vision_{n_students}_{n_groups}_x_min_sum_BQM.pkl", 'wb') as f:
  pickle.dump(outputs, f)

In [ ]:
def load_pickle(filename):
    """
    Load a pickle file and return its contents.

    Parameters:
    filename (str): Path to the pickle file.

    Returns:
    object: The deserialized object from the pickle file.
    """
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
example = load_pickle('Vision_21_3_x_min_sum_BQM.pkl')

In [ ]:
SampleSet.from_serializable(example[0]['sampleset']).record

# Creating BQM model

In [ ]:
LAMBDA = n_groups * tao_max*( tao_max - 1)
print(LAMBDA)

In [ ]:
x = np.array([[f'x_{i}_{c}' for c in C] for i in N])

In [ ]:
bqm = BinaryQuadraticModel(Vartype.BINARY)

In [ ]:
# objective function

for i in N:
  for j in N:
    if i == j or alpha.get((i,j),0) == 0:
      continue
    for c in C:
      var1 = x[i,c]
      var2 = x[j,c]
      cost = -1*alpha.get((i,j),0)
      bqm.add_quadratic(var1,var2,cost)

In [ ]:
# Constraint one single group constraint

for i in N:
    terms = x[i,:]
    terms = [(x,1) for x in terms]
    bqm.add_linear_equality_constraint(terms = terms, lagrange_multiplier = LAMBDA, constant = -1)
    terms.clear

In [ ]:
# min group size

for c in C:
    terms = x[:,c]
    terms = [(x,1) for x in terms]
    bqm.add_linear_inequality_constraint(terms = terms, lagrange_multiplier = LAMBDA, lb = tao_min,ub = tao_max,label = 'tao_min')
    terms.clear

In [ ]:
# max group size

for c in C:
    terms = x[:,c]
    terms = [(x,1) for x in terms]
    bqm.add_linear_inequality_constraint(terms = terms, lagrange_multiplier = LAMBDA, constant = -tao_max,label = 'tao_max')
    terms.clear

In [ ]:
# Beta homegeinity constraints
group_combs = [(c,cp) for c in C for cp in C if c!=cp]

for c1,c2 in group_combs:
  for f in range(n_features):

    bf = beta[f]
    B = beta_max[f]

    terms1 = [(x[i,c1],bf[i]) for i in N]
    terms2 = [(x[i,c2],-bf[i]) for i in N]
    terms = terms1 + terms2
    bqm.add_linear_inequality_constraint(terms = terms, lagrange_multiplier = LAMBDA, constant = -B,label = 'homophily_break')

# Solve the BQM model

In [ ]:
print(len(bqm.linear))
print(len(bqm.quadratic))

In [ ]:
def produce_anneal_schedule(anneal_time,pause_duration):

  step = anneal_time//2

  p1 = [0,0]
  p2 = [step,0.5]
  p3 = [step+pause_duration,0.5]
  p4 = [anneal_time + pause_duration,1.0]

  return [p1,p2,p3,p4]

In [ ]:
anneal_time = 20
pause_duration = 10

schedule = produce_anneal_schedule(anneal_time,pause_duration)
print("Schedule: %s" % schedule)

plt.figure(1, figsize=(3, 3))
plt.plot(*np.array(schedule).T)
plt.title("Anneal Schedule")
plt.xlabel("Time [us]")
plt.ylabel("Annealing Parameter s")
plt.ylim([0.0,1.0])
plt.show()

In [ ]:
sampler = DWaveSampler(solver='Advantage_system6.4',endpoint= endpoint, token = token)
print("Connected to sampler", sampler.solver.name)

In [ ]:
sampler = EmbeddingComposite(sampler)
sampler_name = sampler.properties['child_properties']['chip_id']

In [ ]:
NUM_READS = 1000
CHAIN_S = LAMBDA
outputs = []

anneal_times = [20]
pause_durations = [0,]

total_iterations = len(anneal_times)*len(pause_durations)

with tqdm(total=total_iterations) as pbar:

  for anneal_time in anneal_times:
    for pause_duration in pause_durations:
      schedule = produce_anneal_schedule(anneal_time,pause_duration)
      instance_name = f'R_MN_SAP_BQM_{n_students}_{n_groups}_{anneal_time}_{pause_duration}'
      if pause_duration == 0:
        sampleset = {}
        sampleset = sampler.sample(bqm, num_reads= NUM_READS,annealing_time=anneal_time,label = instance_name,chain_strength=CHAIN_S)
        row = {'n':n_students,'c':n_groups,'anneal_time':anneal_time,'pause_duration':pause_duration,'sampleset':sampleset}
        outputs.append(row)
        pbar.update(1)
      else:
        sampleset = {}
        sampleset = sampler.sample(bqm, num_reads= NUM_READS,label=instance_name,chain_strength=CHAIN_S,anneal_schedule=schedule)
        row = {'n':n_students,'c':n_groups,'anneal_time':anneal_time,'pause_duration':pause_duration,'sampleset':sampleset}
        outputs.append(row)
        pbar.update(1)

In [ ]:
product(anneal_time,pause_durations)

In [ ]:
combs = [(a,b) for a in anneal_times for b in pause_durations]
for comb,out in zip(combs,outputs):
  try:
    print(comb, out['sampleset'].first.energy)
  except:
    print(f'problem with {comb}')

In [ ]:
with open(f"Vision_{n_students}_{n_groups}_sampleset_min_sum_bqm.pkl", 'wb') as f:
  pickle.dump(outputs, f)